In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import idx2numpy
import numpy as np

In [ ]:
# MNIST_DIR = "/content/m_nist/"
MNIST_DIR = "../mnist/"

In [ ]:
train_arr = idx2numpy.convert_from_file(MNIST_DIR + "train-images-idx3-ubyte")
print(train_arr.shape)

In [ ]:
train_dataset_flattened = train_arr.reshape(60000, -1)
print(train_dataset_flattened.shape)

In [ ]:
print(np.max(train_dataset_flattened), np.min(train_dataset_flattened))

In [ ]:
train_dataset_flattened = train_dataset_flattened / 255.0
print(np.max(train_dataset_flattened), np.min(train_dataset_flattened))

FC Auto Encoder


In [ ]:
encoder = tf.keras.Sequential([
    tf.keras.Input(shape=(784,)),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(8, activation="linear")
])

In [ ]:
encoded = encoder(train_dataset_flattened[:5])
print(encoded.shape)

In [ ]:
decoder = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(784, activation="sigmoid")
])

decoded = decoder(encoded)
print(decoded.shape)

In [ ]:
auto_encoder_fc = tf.keras.Sequential([encoder, decoder])
dummy_images = auto_encoder_fc(train_dataset_flattened[:5])
print(dummy_images.shape)

In [ ]:
auto_encoder_fc.summary()

In [ ]:
auto_encoder_fc.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss="mean_squared_error",
    metrics=['accuracy']
)

In [ ]:
history = auto_encoder_fc.fit(
    train_dataset_flattened, train_dataset_flattened,
    batch_size=64,
    epochs=10)

In [ ]:
plt.title("Train Loss")
plt.plot(history.history["loss"])
plt.show()

In [ ]:
def show_images(original_images, decoded_iamges):
    plt.figure(figsize=(6, 6))
    plt.box(False)
    plt.xticks([])
    plt.yticks([])
    img_idx = 0
    for i in np.arange(1, 9, 2):
        original_image = original_images[img_idx].reshape((28, 28))
        decoded_image = decoded_iamges[img_idx].reshape((28, 28))
        plt.subplot(4, 2, i)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(original_image, cmap="gray")
        plt.subplot(4, 2, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(decoded_image, cmap="gray")
        img_idx += 1

    plt.show()

In [ ]:
img_batch = train_dataset_flattened[:4]
decoded_image_batch = auto_encoder_fc.predict(img_batch)
print(img_batch. shape, decoded_image_batch.shape)

In [ ]:
show_images(img_batch, decoded_image_batch)

In [ ]:
auto_encoder_fc.layers

In [ ]:
compressed_images = auto_encoder_fc.layers[0](img_batch)
print(compressed_images)

In [ ]:
restored_images = auto_encoder_fc.layers[1](compressed_images)
show_images(img_batch, decoded_image_batch)

ConvNet Auto Encoder


In [ ]:
print(train_arr.shape)
print(np.max(train_arr), np.min(train_arr))

In [ ]:
train_dataset_2D = train_arr / 255.0
print(train_dataset_2D.shape)
train_dataset_2D = np.expand_dims(train_dataset_2D, axis=3)
print(train_dataset_2D.shape)
print(np.max(train_dataset_2D), np.min(train_dataset_2D))

In [ ]:
encoder_conv = tf.keras.Sequential([
    tf.keras.Input(shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(8, 3, strides=2, activation="relu"),
    tf.keras.layers.Conv2D(16, 3, strides=2, activation="relu"),
    tf.keras.layers.Conv2D(32, 5, activation="linear")
])

dummy_batch = train_dataset_2D[:64]
print(dummy_batch.shape)
dummy_2D_encoded = encoder_conv(dummy_batch)
print(dummy_2D_encoded.shape)

In [ ]:
decoder_conv = tf.keras.Sequential([
    tf.keras.layers.Conv2DTranspose(32, 5, activation="relu"),
    tf.keras.layers.Conv2DTranspose(16, 3, strides=2, activation="relu"),
    tf.keras.layers.Conv2DTranspose(8, 3, strides=2, activation="relu"),
    tf.keras.layers.Conv2DTranspose(1, 2, activation="sigmoid")
])

dummy_2D_decoded = decoder_conv(dummy_2D_encoded)
print(dummy_2D_decoded.shape)

In [ ]:
auto_encoder_conv = tf.keras.Sequential([encoder_conv, decoder_conv])
dummy_images = auto_encoder_conv(dummy_batch)
print(dummy_images.shape)

In [ ]:
auto_encoder_conv.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss="mean_squared_error",
    metrics=['accuracy']
)

history = auto_encoder_conv.fit(
    train_dataset_2D, train_dataset_2D,
    batch_size=64,
    epochs=5)

In [ ]:
plt.title("Train Loss")
plt.plot(history.history["loss"])
plt.show()

In [ ]:
original_images = train_dataset_2D[:64]
restored_images = auto_encoder_conv(original_images)

plt.figure(figsize=(6, 6))
plt.box(False)
plt.xticks([])
plt.yticks([])
img_idx = 0
for i in np.arange(1, 9, 2):
    original_image = original_images[img_idx].squeeze()
    decoded_image = restored_images[img_idx].numpy().squeeze()
    plt.subplot(4, 2, i)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(original_image, cmap="gray")
    plt.subplot(4, 2, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(decoded_image, cmap="gray")
    img_idx += 1

plt.show()